In [ ]:
from flask import *
import numpy as np
from keras.models import load_model
import fasttext
import re
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch



model=load_model('model.h5')

en_model=fasttext.load_model('cc.en.300.bin')

model=T5ForConditionalGeneration.from_pretrained('./finetune-t5')
tokenizer=T5Tokenizer.from_pretrained('t5-base')
prediction_occurrences = {}

app=Flask(__name__,template_folder='templates')


# preprocess text
def preprocess(text):
    process_text=re.compile('[^A-Za-z0-9]')
    text=re.sub(process_text,' ',text.lower())
    return text

# convert text into vectors using Fasttext
def vectorize_text(text):
    vector = en_model.get_sentence_vector(text)
    vector = np.reshape(vector, (1, 1, 300))
    return vector

# generate automated reply for every comment using transformers model
def generate_reply(comment):
    input_ids=tokenizer.encode(comment,return_tensors='pt')
    with torch.no_grad():
        output=model.generate(input_ids,
                             max_length=256, 
                             no_repeat_ngram_size=1, 
                             num_beams=30, 
                             early_stopping=True,
                            )
    reply=tokenizer.decode(output[0],skip_special_tokens=True)
    return reply
   
@app.route('/')
def home(): 
    return render_template('index.html',prediction_occurrences={})   

@app.route('/predict', methods=['GET','POST'])
def predict():
    
    
     global prediction_occurrences;
    
     if request.method == 'POST':
        input_text = request.form['text']
   
        processed_text = preprocess(input_text)

        vectorized_text = vectorize_text(processed_text)

        prediction = model.predict(vectorized_text)

        predicted_class = np.argmax(prediction)

        classes = [ 'Gratitude','About the recipe','About the video','Praising','Hybrid','Undefined','Suggestions and queries']
        result = classes[predicted_class]
        
        prediction_occurrences[result] = prediction_occurrences.get(result, 0) + 1
        
        # automate reply with T5 model if prediction is Gratitude and praising 
        if result in ['Gratitude','Praising','About the recipe','About the video','Hybrid','Undefined']:
            automated_reply=generate_reply(input_text)
            
            return render_template('index.html', prediction=result, automated_reply=automated_reply,
                                   prediction_occurrences=prediction_occurrences)
        
        return render_template('index.html', prediction=result,prediction_occurrences=prediction_occurrences)
          

if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)

In [1]:
import pandas as pd
df=pd.DataFrame({
    'comments':[
        'wooooooo its very yummmmmm I love it',
        'thanx.. respect from',
        'you re amazing',
        'thank u so much ...love u  my friend.Im from Dubai',
        'u r awesome kabita ji',
        'aap mujhe bahut pasand ho mam',
        'mast',
        'very good',
        'i tried it today it came out delicious.. I have added kade masala and other  ingredients 1.5 times more for 2kg chicken and 1 kg 100 gm  basmathi rice and i followed each and every step  it came out really well.. My family loved the recipe.. Thank u for sharing this recipe',
        'very nice kabita sis ,I m in Madurai Tamil nadu',
        'nice presentation',
        'wow nice',
        'u r the best chef on youtube mam',
        'super recipe mam ....I tried it... It was awesome.... Thanks a lot... super you are Keep going',
        'so nice Awesome',
        'Kabita mam, you are the only reason i learned cooking. Not only that i made cooking as my hobby, being a bachelor my roommates always wait for me to come from office and make something for them. All the credit goes to you mam, you are really awesome',
        'i love you darling',
        'wonderful',
        'sister aapka video dekhne ke baad mene life me first time biryani banai, aur jab biryani ban gayi toh gharwale ungliya chatate reh gaye, thank you sister',
        'hello Kabita Ma am ... Aaj Maine Aapne bataya huye tarike se biryani Banayi... Bahut he acchi Bani thi... Thanks for sharing this recepe with us',
        'you are best',
        'So easy n delicious i learned frm u only',
        'tqqq Didi muja reply daynay ka khaliea.',
        'Kabitha mam appki lookg so cuteeeeeee',
        'mam aupka language bahut acha lagta hai',
        'very nice mam Maine banaya bhut testy Bani thanks',
        'you deserve 10M subscribers mam',
        'superb',
        'you are a gem Much love',
        'kabita mam i follow u regularly n ur dishes nvr fail to get me appreciaion from my mother',
        'owowwo Nice one mam',
        'u r tips r really good',
        'i don t like your style',
        'gajab ek no ban',
        'mashallah Bahut Umda', 
        'all credit goes to u',   
        'mem aap bhut acha Khan bante ho',
        'most wonderful and welcome',
        'swaad hi aa gya.. mast bni h biryani.. thanku mam',
        'u r good as always',
        'my god bless your hands thank you mam',
        'fantastic good job',
        'waah kavita di mjaa aa gya. Hats off to u',
        'star***** you are a star in cooking....loyal',
        'bhut sundr ha aap',
        'bahoot achhi',
        'love u a lottttt...may god bless u wdh d entire happiness of dis world',
        'ur SWEET mam',
        'very helpful.. Thanku mam',
        'bahut bahut bahut achha laga',
        'i love your innovativ idea',
        'dear aunty g You deserve an Oscar for cooking from us',
        'mai apki bohot badhi fan hun',
        'great dear',
        'im ur biggest fan kabita ji.. n my mom also',
        'aap k bataney ka tarika bahut acha hey',
        'hats off 2 u',
        'thanx...its very easy',
        'allah ne kya hunar baksha h',
        'jb se apke btaye trike se khana bnane lgi hn mere hubby bahr khana hi chhod diye thhhhnxxx didu',
        'i love your simple preparation..keep going',
        'bohot achha...thank u',
        'beautiful',
        # 'I dont like it at all',
        # 'Jal gaya tha',
        # 'Naaam bada darshan chota not satisfied with this recipe lots of mistake',
        # 'bakwash',
        # 'chiii mene try ki ghr pr  biryani ki naam ki insult h ye plzz guys koi try mt krna ghr pr',
        # 'I dont like this biriyani',
        # 'Lolllll Yeh biryani Hai ya sabji Ese Todhi na Banti Hai pagal biryani',
        # 'badi mushkil h',
        # 'thnxc for vidoes.....kavita mam',
        # 'I hate it',
        # 'I don t like maggi',
        # 'Banne ke bad bahut lalach lag raha hai',
        # 'Nice video',
        # 'Your recipes are always good',
        # 'Mujhe video bahot acchalaga',
        # 'No one like u',  
        # 'Very tasty luv it',  
    ],
    'reply':[
        'im glad to hear you enjoyed it',
        'youre welcome! sending respect right back at ya',
        'thank you so much! That means a lot to me.',
        'youre welcome! sending warm regards back to you',
        'thank you for your kind words! I appreciate your encouragement',
        'shukriya! aapka yeh izhar bahut khushi ka mauka hai.',
        'bilkul koi baat nahi!',
        'thank you',
        'thats fantastic to hear! im thrilled the recipe turned out so well for you and that your family enjoyed it.',
        'thank you so much! Madurai is such a beautiful place',
        'i appreciate the compliment.',
        'i appreciate it',
        'thank you so much, im thrilled to hear that you enjoy my cooking videos',
        'your feedback means a lot to me.',
        'thank you! im glad you think so.',
        'it warms my heart to hear that',
        'i love you too, darling. you mean the world to me',
        'thank you for your kind words',
        'arrey wah, bohot accha hai! biryani banana ek art hai aur tumne pehli baar hi itna achha kiya, mubarak ho! Aur gharwale ki khushi ka koi mukabla nahi',
        'hey, thank you! Bahut achha laga sunke ki aapko biryani pasand aayi. Khushi hui ki aapne try kiya aur pasand aaya. Agar koi aur recipe try karna ho toh mujhse share karna mat bhoolna',
        'thank you for your kind words',
        'thank you!',
        'ji zaroor, apka koi sawal hai ya kuch specific janna chahte hain',
        'arrey, thank you yaar! tum bhi bilkul cute ho',
        'dhanyavad! Mujhe khushi hai ki aapko meri bhasha pasand aayi.',
        'thank you! Aapka yeh feedback bahut achha laga',
        'i truly appreciate your support and encouragement',
        'thank you for the compliment',
        'sending much love right back to you',
        'im delighted to hear that my recipes have been a hit with you and your mother',
        'im glad you liked it',
        'im glad you find my tips helpful',
        'i appreciate your feedback. is there something specific you d like to see differently, or any suggestions you have in mind',
        'arre bhai, bilkul ek no! thanks yaar, tumne dil khush kar diya',
        'mashallah, bahut shukriya! aapka pyaar aur sammaan hamesha ke liye yaad rahega',
        'i really appreciate your acknowledgment',
        'dhanyavaad! Main khana banane mein dil se laga hoon',
        'thank you very much!',
        'bohot khushi hui ki biryani pasand aayi! aapka swagat hai, anytime',
        'thank you! I m here to help whenever you need assistance',
        'thank you so much for your kind words! its my pleasure to help',
        'your appreciation fuels my dedication',
        'shukriya! aapka pyaar aur samman hamesha muskurahat ka saathi banata hai.',
        'thank you for the stellar compliment',
        'shukriya! Aapka yeh pyara sandesh mere dil ko choo gaya',
        'kya baat hai, dil khush kar diya',
        'your affection fills my heart with warmth and joy',
        'thank you for your kindness',
        'youre welcome! glad i could assist',
        'yeh sunke mujhe bhi bahut khushi hui',
        'your appreciation fuels my passion for innovation even more',
        'thank you so much for the heartfelt compliment',
        'wow, sach? Yeh toh amazing hai! Dil se shukriya, yaar',
        'thanks!',
        'im truly grateful to have fans like you and your mom',
        'dhanyavaad! aapki tareef sun kar khushi hu',
        'thank you! i tip my hat right back at you',
        'glad I could make it straightforward for you',
        'allah ne mujhe pyaar dena ka khaas hunar baksha hai',
        'arrey wah! bahut khushi hui sunke! Ab toh apka ghar ka khana hi sabka dil jeet raha hai na',
        'thank you! Ill continue to keep things straightforward and enjoyable',
        'bahut shukriya',
        'thank you',
        # 'I appreciate your honesty. Is there something we can do to make it better for you',
        # 'Arre bhai, samajh gaya. Koi baat nahi, agli baar dekhte hain kaise kar sakte hain.',
        # 'Understood, we all make the necessary adjustments',
        # 'maafi yaar. Next time behtar koshish karenge',
        # 'Samajh gaya, shayad kuch gadbad ho gayi hogi. Aage se try nahi karenge, koi baat nahi.',
        # 'I understand. We will make sure to improve it next time',
        # 'maafi yaar. Lagta hai kuch gadbad ho gayi. Agli baar behtar try karenge',
        # 'kuch cheezein thodi mushkil hoti hain. Par hum try karenge aur behtar karenge.',
        # 'Youre welcome! Im glad you found the videos helpful',
        # 'I understand. Thank you for sharing your honest opinion',
        # 'Got it, noted. We ll keep that in mind for future reference',
        # 'khane ke baad lalach toh hota hi hai. Par hosh mein raho, control karo',
        # 'Thank you! Im glad you enjoyed the video',
        # 'Thank you for the compliment! I strive to create unique and delicious recipes',
        # 'maza aaya! Agli baar bhi dekhenge.',
        # 'Thank you for your feedback. I will strive to improve and hopefully win you over in the future.',
        # 'Im glad you enjoyed it! Your appreciation means a lot.'
    ]
})

df=df.to_csv('new_data.csv',index=False)

In [2]:
df=pd.read_csv('./new_data.csv')

In [ ]:
df

In [3]:
# import library
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import Dataset
from transformers import DataCollatorWithPadding
import warnings
warnings.filterwarnings('ignore')

In [4]:
#split data into train and test
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = dataset['train']
test_dataset = dataset['test']


In [ ]:
train_dataset.to_pandas()

In [5]:
# intialize models
MODEL_NAME = 't5-base' #'t5-small'#'t5-base'
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# preprocess the data
def pre_process(example):
    data={}
    data['prompt']=[f"comments: {comment}" for comment in example["comments"]]
    
    data['response']=[f'reply:{reply}' for reply in example['reply']]
    
    inputs = tokenizer(data['prompt'], padding='max_length', truncation=True, max_length=512)
    targets = tokenizer(data['response'], padding='max_length', truncation=True, max_length=256)

    inputs.update({'labels': targets["input_ids"]})
    return inputs

In [7]:
# create dataset train and test
train_dataset = train_dataset.map(pre_process, batched=True)
test_dataset = test_dataset.map(pre_process, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,return_tensors='pt')

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

In [ ]:
train_dataset.to_pandas()

In [8]:
#train model here
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

TRAINING_OUTPUT = './result' #r"D:/GenAI/T5/health_samp"
training_args = TrainingArguments(
    output_dir=TRAINING_OUTPUT,
    num_train_epochs=7,
    per_device_train_batch_size=6,  # Lower batch size for T5 models
    per_device_eval_batch_size=6,   # Lower batch size for T5 models
    save_strategy="epoch",
    learning_rate=0.001,
    save_steps=10000,
    max_grad_norm=0.9,
    eval_steps=1000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=test_dataset,  
)
trainer.train()

  0%|          | 0/63 [00:00<?, ?it/s]

Checkpoint destination directory ./result\checkpoint-9 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./result\checkpoint-18 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./result\checkpoint-27 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./result\checkpoint-36 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./result\checkpoint-45 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./result\checkpoint-54 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./result\checkpoint-63 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'train_runtime': 2566.7913, 'train_samples_per_second': 0.136, 'train_steps_per_second': 0.025, 'train_loss': 0.5932701353042845, 'epoch': 7.0}


TrainOutput(global_step=63, training_loss=0.5932701353042845, metrics={'train_runtime': 2566.7913, 'train_samples_per_second': 0.136, 'train_steps_per_second': 0.025, 'train_loss': 0.5932701353042845, 'epoch': 7.0})

In [9]:
#save model
trainer.save_model('./finetune-t5')

In [ ]:
# trainer.save_model('./finetuned-t5')

In [10]:
#initialize fine tuned model
model1 = T5ForConditionalGeneration.from_pretrained('./finetune-t5')
tokenizer=T5Tokenizer.from_pretrained('t5-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# function for generate reply using encoders and decoders
def generate_reply(text):
    inputs = tokenizer( text, return_tensors='pt', max_length=256, padding='max_length', truncation=True)
    outputs = model1.generate(inputs['input_ids'],
                             max_length=256, 
                             no_repeat_ngram_size=1, 
                             num_beams=30, 
                             early_stopping=True,
                            )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [12]:
# result for automated reply
comment = "Very tasty luv it" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:your feedback means a lot to me.


In [13]:
comment = "Aapka beta bahut cute h love you beta" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:bahut shukriya! You mean the world to me


In [14]:
comment = "Wao mam thanks Mene banae ye recipe...it's so yamm" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:thank you for your kind words


In [15]:
comment = "super excited thanku" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:your appreciation means a lot to me.


In [17]:
comment = "Boht badiya mam thanks" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:bahut shukriya


In [20]:
comment = "You're amazing" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:thank you so much! I appreciate your encouragement


In [23]:
comment = "Ap boht achi hain our apki recipe bhi kamal  ki hai" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:arrey, thank you so much! Apki raha hai.


In [25]:
comment = "Your cooking is so neat and clean" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:your feedback means a lot to me.


In [26]:
comment = "i like your style" 
reply = generate_reply(comment)

print("Generated Reply:", reply)

Generated Reply: reply:youre welcome! glad you liked it


In [ ]:
inp = test_dataset['comments']
oup = test_dataset['reply']

inp[7],oup[7]

In [ ]:
print(generate_reply(oup[6]))

In [ ]:
print(generate_reply('Tqqq Didi muja reply daynay ka khaliea'))